<a href="https://colab.research.google.com/github/haritanair/IMDB-Sentiment-Analysis/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Import necessary libraries
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import nltk

# Download stopwords and WordNet Lemmatizer (ensure you have NLTK data)
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Load the dataset
from google.colab import files
uploaded =files.upload()




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Saving IMDB Dataset.csv to IMDB Dataset.csv


In [ ]:
df=pd.read_csv("IMDB Dataset.csv")

# Show the first few rows of the dataset
df.head()



,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from collections import Counter
import contractions
!pip install emoji
import emoji
from sklearn.model_selection import train_test_split
# Download the 'averaged_perceptron_tagger'
import nltk
nltk.download('averaged_perceptron_tagger')

# Step 1: Handle Missing Data
df.dropna(inplace=True)

# Step 2: Remove Duplicates
df.drop_duplicates(inplace=True)

# Step 3: Expand Contractions
def expand_contractions(text):
    return contractions.fix(text)

# Step 4: Handle Emojis
def handle_emojis(text):
    return emoji.demojize(text)

# Step 5: Normalize Repeated Characters
def normalize_repeated_chars(text):
    return re.sub(r'(.)\1{2,}', r'\1\1', text)  # Reduce repeated characters to two

# Step 6: Clean the text
def clean_text(review):
    # Lowercasing
    review = review.lower()
    # Expand contractions
    review = expand_contractions(review)
    # Remove HTML tags
    review = BeautifulSoup(review, "html.parser").get_text()
    # Handle emojis
    review = handle_emojis(review)
    # Remove punctuation and special characters
    review = re.sub(r'[^a-zA-Z\s]', '', review)
    # Normalize repeated characters
    review = normalize_repeated_chars(review)

    # Tokenization
    words = word_tokenize(review)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words and len(word) > 1]

    # Handle Negations
    words = [word.replace("not", "not_") for word in words]  # For example, "not good" becomes "not_good"

    # Lemmatization with POS tagging
    pos_tags = pos_tag(words)
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word, pos='v') for word, pos in pos_tags]  # Verb lemmatization

    return ' '.join(words)

# Apply the cleaning function to the entire dataset
df['cleaned_review'] = df['review'].apply(clean_text)

# Show the first few cleaned reviews
print(df[['review', 'cleaned_review']].head())

# Step 7: Remove Rare Words
word_counts = Counter(" ".join(df['cleaned_review']).split())
rare_words = set([word for word, count in word_counts.items() if count < 5])
df['cleaned_review'] = df['cleaned_review'].apply(lambda x: ' '.join([word for word in x.split() if word not in rare_words]))

# Step 8: Prepare for Modeling
X = df['cleaned_review']
y = df['sentiment']  # Assuming your sentiment column is named 'sentiment'
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Optional: Save the cleaned data
df.to_csv('cleaned_IMDB_Dataset.csv', index=False)

# Future Steps for Modeling
# Here you can use the cleaned data with models like BERT, LSTM, GRU, or CNN.
# For BERT, you would use the Hugging Face Transformers library.
# For LSTM/GRU/CNN, you would prepare the data into sequences and pad them accordingly.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
<ipython-input-4-2dc59aac2078>:43: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review = BeautifulSoup(review, "html.parser").get_text()


                                              review  \
0  One of the other reviewers has mentioned that ...   
1  A wonderful little production. <br /><br />The...   
2  I thought this was a wonderful way to spend ti...   
3  Basically there's a family where a little boy ...   
4  Petter Mattei's "Love in the Time of Money" is...   

                                      cleaned_review  
0  one reviewers mention watch oz episode hook ri...  
1  wonderful little production film technique una...  
2  think wonderful way spend time hot summer week...  
3  basically family little boy jake think zombie ...  
4  petter matteis love time money visually stun f...  


In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Map string labels to integers (1 for positive, 0 for negative)
label_mapping = {'positive': 1, 'negative': 0}
df['label'] = df['sentiment'].map(label_mapping)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['label'], test_size=0.2, random_state=42)

# Function to clean the text
def clean_text(review):
    review = review.lower()
    review = re.sub(r'[^a-zA-Z\s]', '', review)  # Remove punctuation
    return review

# Clean the reviews
X_train = X_train.apply(clean_text)
X_test = X_test.apply(clean_text)

# Tokenization
max_words = 5000  # Maximum number of words to keep, based on frequency
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to ensure uniform input size
max_len = 100  # Maximum length of sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

### 2. **Define Models**

def create_lstm_model():
    model = Sequential()
    model.add(Embedding(input_dim=max_words, output_dim=100, input_length=max_len))
    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(64))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model

def create_gru_model():
    model = Sequential()
    model.add(Embedding(input_dim=max_words, output_dim=100, input_length=max_len))
    model.add(GRU(128, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(GRU(64))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model

def create_cnn_model():
    model = Sequential()
    model.add(Embedding(input_dim=max_words, output_dim=100, input_length=max_len))
    model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model


In [ ]:
# Function to train and evaluate a model
def train_and_evaluate_model(model):
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_loss', patience=3)
    history = model.fit(X_train_pad, y_train,
                        epochs=5,
                        batch_size=32,
                        validation_split=0.2,
                        callbacks=[early_stopping],
                        verbose=0)  # Set verbose to 1 if you want to see training progress
    loss, accuracy = model.evaluate(X_test_pad, y_test, verbose=0)
    return accuracy

# Create and evaluate LSTM model
lstm_model = create_lstm_model()
lstm_accuracy = train_and_evaluate_model(lstm_model)
print(f'LSTM Model Accuracy: {lstm_accuracy:.4f}')



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


LSTM Model Accuracy: 0.8538


In [ ]:
# Create and evaluate GRU model
gru_model = create_gru_model()
gru_accuracy = train_and_evaluate_model(gru_model)
print(f'GRU Model Accuracy: {gru_accuracy:.4f}')



GRU Model Accuracy: 0.8452


In [ ]:
# Create and evaluate CNN model
cnn_model = create_cnn_model()
cnn_accuracy = train_and_evaluate_model(cnn_model)
print(f'CNN Model Accuracy: {cnn_accuracy:.4f}')


CNN Model Accuracy: 0.8469
